# PseudoCode Lorentz Workshop eWaterCycle
This notebook shows the type of tutorial notebook we want to present to the participants of the Lorentz workshop

In [ ]:
from ewatercycle.models import WflowJl, Sfincs
from ewatercycle.base.forcing import DefaultForcing
from ewatercycle.base.parameter_set import ParameterSet
from ewatercycle.observation.grdc import get_grdc_data

# Case study: Humber
We present a coupling of a hydrologic and a hydrodynamic model with data prepared for the Humber estuary as a working workflow to start our workshop with. This will be the base we use to develop other use cases / workflows.

In [ ]:
#some additional settings, 
experiment_start_date = "1990-01-01T00:00:00Z"
experiment_end_date = "2000-12-31T00:00:00Z"

In [ ]:
shapefile = "/some/location/humber.shp"
grcd_station_id = "12345"

discharge_observation, _ = get_grdc_data(
    grcd_station_id,
    start_time=experiment_start_date,
    end_time=experiment_end_date,
)

In [3]:
#some settings on how the coupling is achieved
discharge_coupling_locations = {
    "locationName1": {"lat": 1.32532153, "lon": 52.1351345},
    "locationName2": {"lat": 0.32532153, "lon": 51.1641345},
}

lats = [val["lat"] for _, val in discharge_coupling_locations.items()]
lons = [val["lons"] for _, val in discharge_coupling_locations.items()]

Create forcing (ie. temperature and rainfall data) for the hydrological model

In [ ]:
#we already prepared a forcing file, which we can load with this command. The forcing was generated using this commented code.

#experiment_forcing = ewatercycle.forcing.generate(
#    target_model='wflowjl', 
#    dataset="ERA5"
#    start_time=experiment_start_date,
#    end_time=experiment_end_date, 
#    shape=shapefile, 
#)

wflow_humber_forcing = DefaultForcing.load("location of prepared forcing data")
sfincs_humber_forcing = DefaultForcing.load("location of prepared forcing data")

Each model needs parameters sets. Often the combination of model and parameter set is actually what is called "model". At Deltares, the combination of parameters for the humber and the wflow model would be called the "wflow humber model". In the context of eWaterCycle, we decided to call this a model instance, to make a distinction between the model as a piece of software and the model instance as a combination of model and data.

In [ ]:
#The WFlow parameter set for the humber is derived using the HydroMT software made by Deltares.
wflow_humber_parameters = ParameterSet(
    name="wflow-humber",
    directory="location of wflow humber parameters",
    config="dir/config.yaml"
)

#The Sfincs parameter set is prepared by Tycho and given to us
sfincs_humber_parameters = ParameterSet(
    name="sfincs-humber",
    directory="location of wflow humber parameters",
    config="dir/config.yaml"
)

Create the model instances and initialize the models

In [ ]:
hydrologicModel = WflowJl(
    version="valid_version_nr",  # ewatercycle.models.WflowJl.available_versions
    forcing=wflow_humber_forcing,
    parameter_set=wflow_humber_parameters
)

In [ ]:
hydrodynamicModel = Sfincs(
    version="valid_version_nr",  # ewatercycle.models.Sfincs.available_versions
    forcing=sfincs_humber_forcing,
    parameter_set=sfincs_humber_parameters
)

In [ ]:
hydrologic_cfg_file, hydrologic_cfg_dir = hydrologicModel.setup()

In [ ]:
hydrodynamic_cfg_file, hydrodynamic_cfg_dir = hydrodynamicModel.setup()

In [ ]:
hydrologicModel.initialize(hydrologic_cfg_file)

In [ ]:
hydrodynamicModel.initialize(hydrodynamic_cfg_file)

Get some info on the models so

In [ ]:
print(hydrologicModel.get_output_vars())
print(hydrologicModel.get_input_vars())
print(hydrodynamicModel.get_output_vars())
print(hydrodynamicModel.get_input_vars())

### Run models & capture output
This is the main loop of an eWaterCycle experiment. As long as the time variable of the model has not surpassed the end time of the experiment, `model.update()` runs a single timestep of a model. the `simulated_output` and `timestamps` arrays are filled with the calculated output of interest and timestamps respectivly using `model.get_value()` and `model.time`.

Coupling is achieved by taking the values of 'discharge' from the hydrologic model at certain locations and setting them using the `model.set_value_at_ind()` function into the hydrodynamic model.

In [ ]:
simulated_output = []
timestamps = []

while (hydrologicModel.time < hydrologicModel.end_time):

    # update the hydrological model. The order of updating and exchanging information is 
    # very dependent on the definition of the information exchanged. Here it is assumed that
    # the discharge calculated by the hydrological model represents the average discharge over the timestep just
    # finished. This is given to the hydrodynamic model as (upstream) boundary condition
    hydrologicModel.update()

    #get discharge from hydrological model at the locations of interest
    #NOTE: the name of the variable is made up!
    exchange_items = hydrologicModel.get_value_at_coords(
        "discharge",
        lat=lats,
        lon=lons,
    )
    #set the boundary condition value in the hydrodynamic model.
    #NOTE: the name of the variable is made up!
    hydrodynamicModel.set_value_at_coords(
        "upstreamDischarge", 
        lat=lats,
        lon=lons,
        values=exchange_items,
    )

    #update the hydrodynamic model. Because we don't assume that the models run the same timestep, we use 
    #model.update_until()

    hydrodynamicModel.update_until(hydrologicModel.time)

    #get output of interest from the hydrdynamic model
    
    timestamps.append(hydrodynamicModel.time)
    simulated_output.append(hydrodynamicModel.get_value("varOfInterest"))

The `timestamps` and `simulated_output` variables are now filled with datetime values and outputs for each simulated day.

### Clean up after the model run
The models have to be 'finalized', which deletes any temporary files and the containers have to be shut down.

In [ ]:
hydrodynamicModel.finalize()
hydrologicModel.finalize()

### Plot the results

Combine simulated and observated discharge into a single dataframe

In [ ]:
import pandas as pd

simulated_output_df = pd.DataFrame(
    {'simulation': simulated_output}, index=pd.to_datetime(timestamps)
)
simulated_output_df.plot()
discharge_observation.plot()